<a href="https://colab.research.google.com/github/MILLERMARRU/numero_cnn/blob/main/numeros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit streamlit-drawable-canvas transformers torch torchvision pyngrok




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 388.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [2]:
!ngrok config add-authtoken 2rPOA88C1NvlhE7pXulxY6VlUMA_4LMGUuiywutna8pigaZ4A

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
code = """

import streamlit as st
from streamlit_drawable_canvas import st_canvas
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image, ImageOps
import torch
import numpy as np
import pandas as pd

st.set_page_config(page_title="Reconocedor MNIST (ViT)", layout="centered")
st.title('✏️ Dibuja un número (0‑9) y pulsa "Reconocer"')


@st.cache_resource
def cargar_modelo():
    model_id = "farleyknight/mnist-digit-classification-2022-09-04"
    processor = AutoImageProcessor.from_pretrained(model_id)
    model = AutoModelForImageClassification.from_pretrained(model_id)
    model.eval()
    return processor, model

processor, model = cargar_modelo()

canvas = st_canvas(
    fill_color="#000000", stroke_width=10, stroke_color="#FFFFFF",
    background_color="#000000", width=280, height=280,
    drawing_mode="freedraw", key="canvas"
)

if st.button("🔍 Reconocer número"):
    if canvas.image_data is not None:
        img = Image.fromarray(canvas.image_data[:, :, 0].astype("uint8"), mode="L")
        img = ImageOps.invert(img).resize((224, 224)).convert("RGB")

        inputs = processor(images=img, return_tensors="pt")
        with torch.no_grad():
            logits = model(**inputs).logits.squeeze()
            probs = torch.softmax(logits, dim=0).numpy()
            pred = int(np.argmax(probs))

        st.success(f"✅ Número reconocido: **{pred}**")
        df = pd.DataFrame({
            "Número": list(range(len(probs))),
            "Probabilidad": np.round(probs, 3)
        }).sort_values("Probabilidad", ascending=False)
        st.table(df)
    else:
        st.warning("¡Dibuja primero antes de reconocer!")




"""
with open("streamlit_app.py", "w") as f:
    f.write(code)


In [4]:
from pyngrok import ngrok
import threading
import time
import os

# Abrir túnel en el puerto 8501
public_url = ngrok.connect(addr="http://localhost:8501")
print("🔗 URL pública:", public_url)

# Función para correr streamlit
def run_streamlit():
    os.system("streamlit run streamlit_app.py")

# Ejecutar streamlit en segundo plano
thread = threading.Thread(target=run_streamlit)
thread.start()

# Espera un poco para que Streamlit inicie
time.sleep(5)



🔗 URL pública: NgrokTunnel: "https://daa28e4f38ff.ngrok-free.app" -> "http://localhost:8501"
